In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import tensorflow as tf

In [2]:
sess = tf.InteractiveSession()

In [2]:
def make_data():
    x = np.random.random(10).reshape(1, -1)
    y = np.zeros((1, 2))
    y[0, np.random.randint(2)] = 1
    return {'features': x, 'label': y}

In [3]:
make_data()

{u'features': array([[ 0.44012259,  0.2345955 ,  0.06387658,  0.12333694,  0.69836099,
          0.8420227 ,  0.77486622,  0.725308  ,  0.29518256,  0.43742561]]),
 u'label': array([[ 1.,  0.]])}

In [4]:
import math
math.ceil(2.5)

3.0

# Random Split

In [45]:
np.random.random(5)

array([ 0.34366436,  0.45424886,  0.2222669 ,  0.77494351,  0.14536613])

In [66]:
data = sc.parallelize(range(10000)).map(lambda i: make_data())

In [67]:
data.persist()

PythonRDD[199] at RDD at PythonRDD.scala:48

In [70]:
epoches = 2
batch_size = 128
data_cnt = data.count()
batch_ratios = [batch_size / data_cnt] * int(math.ceil(data_cnt / batch_size))

In [71]:
sum(batch_ratios)

1.011200000000001

In [72]:
len(batch_ratios)

79

In [99]:
def concate_data(x, y):
    dataset = {}
    for f in ['features', 'label']:
        dataset[f] = np.concatenate((x[f], y[f]), axis=0)
    return dataset


def next_batch_rdd_maker(data_rdd, batch_size, epoch_limit=None):
    data_cnt = data_rdd.count()
    per_batch_ratios = [batch_size / data_cnt] * int(math.ceil(data_cnt / batch_size))
    current_epoch = 0
    def next_batch():
        while True:
            current_epoch += 1
            if epoch_limit & current_epoch > epoch_limit: 
                break 
            batches = data_rdd.randomSplit(per_batch_ratios)
            for batch in batches:
                dataset = batch.reduce(concate_data)
                yield dataset['features'], dataset['label']
    return next_batch

In [96]:
data.cache()
next_batch = next_batch_rdd_maker(data, batch_size)

i = 0
for X_batch, Y_batch in next_batch():
    print(X_batch.shape, Y_batch.shape)
    print(X_batch[0:5])
    print(Y_batch[0:5])
    i += 1
    if i >= 5:
        break

(144, 10) (144, 2)
[[ 0.24224284  0.16498114  0.97234312  0.53917806  0.80139095  0.6658507
   0.40992413  0.02788322  0.58027422  0.46947588]
 [ 0.20606814  0.48826739  0.09488544  0.88665485  0.30190824  0.57492528
   0.87088355  0.25162252  0.11831585  0.84164673]
 [ 0.9526457   0.26640934  0.20788505  0.96359236  0.41072184  0.57184716
   0.21038411  0.09211746  0.08085801  0.47561801]
 [ 0.09000604  0.9915175   0.64065629  0.68570067  0.98857309  0.28390843
   0.74060443  0.8019072   0.35338927  0.19615802]
 [ 0.56263761  0.27118114  0.49624215  0.38543442  0.9920974   0.29561692
   0.9599687   0.54038119  0.30061586  0.52189192]]
[[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
(135, 10) (135, 2)
[[ 0.76534103  0.37355596  0.10765203  0.25866323  0.93718641  0.66272085
   0.55086142  0.06528591  0.93724369  0.18067686]
 [ 0.63557297  0.17582746  0.7325152   0.64186083  0.4591681   0.26556179
   0.26458584  0.37710498  0.39643202  0.9728941 ]
 [ 0.87850419  0.85969076  0.13

In [60]:
data.cache()

for epoch in range(epoches):
    batches = data.randomSplit(batch_ratios)
    i = 0
    for batch in batches:
        print("epoch:", epoch, "iteration:", i)
        dataset = batch.reduce(lambda x, y: np.concatenate((x, y), axis=0))
        print(dataset[0:5])
        i += 1 

epoch: 0 iteration: 0
[[ 0.47079038  0.44127603  0.34198059  0.29705626  0.99085371]
 [ 0.29086968  0.8212924   0.93312338  0.52706642  0.39311667]
 [ 0.88315406  0.31705374  0.27846729  0.63873344  0.78870002]
 [ 0.97544966  0.67234266  0.55680115  0.68888139  0.2370686 ]
 [ 0.30005052  0.55812249  0.3856886   0.32348831  0.60008071]]
epoch: 0 iteration: 1
[[ 0.25759934  0.02125317  0.68177384  0.53266375  0.70909405]
 [ 0.61402266  0.95378653  0.21160353  0.04461876  0.66389214]
 [ 0.28667079  0.70980975  0.62265334  0.90745029  0.77433429]
 [ 0.34473192  0.41749686  0.37605562  0.08639847  0.36566433]
 [ 0.6714306   0.14515143  0.41088539  0.74645513  0.6248666 ]]
epoch: 0 iteration: 2
[[ 0.17518729  0.16490345  0.57006935  0.39865782  0.47576799]
 [ 0.16470649  0.49247557  0.93179686  0.76593812  0.31278603]
 [ 0.78117413  0.03550931  0.85237812  0.40126105  0.96470968]
 [ 0.49252665  0.85205034  0.11661615  0.61780545  0.2801653 ]
 [ 0.64048368  0.10050192  0.01868673  0.40520968 

# Local Iterator

In [63]:
rdd = sc.parallelize(range(10000))

In [64]:
for i in rdd.toLocalIterator():
    print(i)
    if i  >= 20:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


# MapPartition

In [5]:
data_rdd = sc.parallelize(range(100000)) \
    .map(lambda i: make_data())

In [6]:
data_rdd.take(1)

[{u'features': array([[ 0.79552556,  0.62010459,  0.12315849,  0.60536968,  0.91883181,
           0.76100623,  0.62098309,  0.78398251,  0.64149226,  0.04521085]]),
  u'label': array([[ 0.,  1.]])}]

In [7]:
data_count = data_rdd.count()
batch_size = 50
n_batch = data_count // batch_size

In [8]:
data_count, n_batch

(100000, 2000)

In [27]:
batch_rdd = data_rdd \
    .coalesce(n_batch, shuffle=True) \
    .zipWithIndex() \
    .map(lambda x: (x[1]%n_batch , x[0])) \
    .groupByKey() \
    .map(lambda x: x[1])

In [28]:
for x in batch_rdd.take(1)[0]:
    print(x)

{u'features': array([[ 0.37073191,  0.48244528,  0.28461215,  0.1317227 ,  0.87890136,
         0.01102265,  0.7651571 ,  0.25752815,  0.75159029,  0.64502424]]), u'label': array([[ 0.,  1.]])}
{u'features': array([[ 0.4504058 ,  0.66501192,  0.2962715 ,  0.02181195,  0.65113811,
         0.90853561,  0.097354  ,  0.58486196,  0.18357662,  0.76886389]]), u'label': array([[ 1.,  0.]])}
{u'features': array([[ 0.95342291,  0.4535336 ,  0.35981249,  0.50075611,  0.99902678,
         0.6014667 ,  0.53381058,  0.48165125,  0.92430657,  0.05714938]]), u'label': array([[ 0.,  1.]])}
{u'features': array([[ 0.52305435,  0.41638638,  0.71556628,  0.70230437,  0.09229949,
         0.92923604,  0.30086101,  0.4653124 ,  0.64653215,  0.21608639]]), u'label': array([[ 1.,  0.]])}
{u'features': array([[ 0.01223647,  0.24719584,  0.86281775,  0.45068686,  0.07528193,
         0.89202074,  0.76245591,  0.34326625,  0.34497815,  0.65970768]]), u'label': array([[ 0.,  1.]])}
{u'features': array([[ 0.33977

In [24]:
batch_rdd.getNumPartitions()

2000

In [31]:
batch_rdd.take(1)

[{u'features': array([[ 0.93172863,  0.98480599,  0.95741531,  0.6129436 ,  0.19673222,
           0.05387264,  0.63988158,  0.11117587,  0.54039591,  0.04718286]]),
  u'label': array([[ 0.,  1.]])}]

In [36]:
def concat_data(it):
    dataset = None
    for d in it:
        if dataset is None:
            dataset = d
        else:
            for f in ['features', 'label']:
                dataset[f] = np.concatenate((dataset[f], d[f]), axis=0)
    yield dataset

batch_rdd.mapPartitions(concat_data).filter(lambda x: x is not None).collect()

[{u'features': array([[ 0.93172863,  0.98480599,  0.95741531,  0.6129436 ,  0.19673222,
           0.05387264,  0.63988158,  0.11117587,  0.54039591,  0.04718286]]),
  u'label': array([[ 0.,  1.]])},
 {u'features': array([[ 0.31638226,  0.85218441,  0.55351473,  0.15053392,  0.09371248,
           0.20815976,  0.86655822,  0.61954134,  0.37148639,  0.92871957],
         [ 0.01892403,  0.39535147,  0.0542648 ,  0.37229283,  0.51681396,
           0.19004711,  0.27305341,  0.32618019,  0.02619873,  0.14676534]]),
  u'label': array([[ 1.,  0.],
         [ 1.,  0.]])},
 {u'features': array([[  3.20721424e-01,   8.64023243e-01,   8.06594026e-01,
            7.97330553e-01,   7.52700299e-01,   7.73324399e-01,
            4.98287828e-01,   9.94801137e-01,   9.96904925e-01,
            2.22312982e-01],
         [  6.77889180e-01,   1.70671847e-01,   6.40587954e-01,
            3.13694353e-01,   5.06828399e-01,   4.71038664e-01,
            4.23924298e-01,   7.47247740e-01,   1.80615691e-01,
  